In [45]:
import pandas as pd
import numpy as np


In [46]:
df = pd.read_csv("theta_gpu.csv")
print(df.head)
print(df.columns)
df["cpu_num"] = 0
df["state"] = 0

<bound method NDFrame.head of                 JOB_NAME  COBALT_JOBID MACHINE_NAME     QUEUED_TIMESTAMP  \
0      10120170.thetagpu      10120170     thetagpu  2022-12-31 23:26:37   
1      10120171.thetagpu      10120171     thetagpu  2022-12-31 23:30:37   
2      10120172.thetagpu      10120172     thetagpu  2023-01-01 00:16:21   
3      10120173.thetagpu      10120173     thetagpu  2023-01-01 00:56:12   
4      10120174.thetagpu      10120174     thetagpu  2023-01-01 01:08:22   
...                  ...           ...          ...                  ...   
22601  10144882.thetagpu      10144882     thetagpu  2023-07-28 07:13:27   
22602  10144883.thetagpu      10144883     thetagpu  2023-07-28 07:29:31   
22603  10144858.thetagpu      10144858     thetagpu  2023-07-28 01:34:22   
22604  10144764.thetagpu      10144764     thetagpu  2023-07-27 10:58:44   
22605  10144860.thetagpu      10144860     thetagpu  2023-07-28 01:51:28   

       QUEUED_DATE_ID      START_TIMESTAMP  START_DATE_ID

In [47]:
df = df[['JOB_NAME', 'USERNAME_GENID', 'PROJECT_NAME_GENID', 'state', 'CORES_USED', 'cpu_num', 'NODES_USED', 'QUEUED_TIMESTAMP', 'START_TIMESTAMP', 'RUNTIME_SECONDS', 'WALLTIME_SECONDS', 'EXIT_STATUS']]

In [48]:
df.head()

,JOB_NAME,USERNAME_GENID,PROJECT_NAME_GENID,state,CORES_USED,cpu_num,NODES_USED,QUEUED_TIMESTAMP,START_TIMESTAMP,RUNTIME_SECONDS,WALLTIME_SECONDS,EXIT_STATUS
0,10120170.thetagpu,2.416635e+13,3.351006e+13,0,1.0,0,1.0,2022-12-31 23:26:37,2022-12-31 23:26:42,2305.54,3600.0,0
1,10120171.thetagpu,NaN,NaN,0,1.0,0,1.0,2022-12-31 23:30:37,2022-12-31 23:30:43,3602.30,3600.0,255
2,10120172.thetagpu,2.416635e+13,3.351006e+13,0,1.0,0,1.0,2023-01-01 00:16:21,2023-01-01 00:16:22,2300.64,3600.0,0
3,10120173.thetagpu,2.416635e+13,3.351006e+13,0,1.0,0,1.0,2023-01-01 00:56:12,2023-01-01 00:56:13,2307.36,3600.0,0
4,10120174.thetagpu,NaN,NaN,0,1.0,0,1.0,2023-01-01 01:08:22,2023-01-01 01:08:22,3603.24,3600.0,255


In [49]:
df["JOB_NAME"] = df["JOB_NAME"].str.split(".").str[0]

In [50]:
df.head()

,JOB_NAME,USERNAME_GENID,PROJECT_NAME_GENID,state,CORES_USED,cpu_num,NODES_USED,QUEUED_TIMESTAMP,START_TIMESTAMP,RUNTIME_SECONDS,WALLTIME_SECONDS,EXIT_STATUS
0,10120170,2.416635e+13,3.351006e+13,0,1.0,0,1.0,2022-12-31 23:26:37,2022-12-31 23:26:42,2305.54,3600.0,0
1,10120171,NaN,NaN,0,1.0,0,1.0,2022-12-31 23:30:37,2022-12-31 23:30:43,3602.30,3600.0,255
2,10120172,2.416635e+13,3.351006e+13,0,1.0,0,1.0,2023-01-01 00:16:21,2023-01-01 00:16:22,2300.64,3600.0,0
3,10120173,2.416635e+13,3.351006e+13,0,1.0,0,1.0,2023-01-01 00:56:12,2023-01-01 00:56:13,2307.36,3600.0,0
4,10120174,NaN,NaN,0,1.0,0,1.0,2023-01-01 01:08:22,2023-01-01 01:08:22,3603.24,3600.0,255


In [51]:
df["USERNAME_GENID"] = df["USERNAME_GENID"].fillna(0)
df["PROJECT_NAME_GENID"] = df["PROJECT_NAME_GENID"].fillna(0)

In [52]:
import datetime

def date_to_unix_seconds(time_string):
  datetime_object = datetime.datetime.strptime(time_string, '%Y-%m-%d %H:%M:%S')
  return datetime_object.timestamp()

df["QUEUED_TIMESTAMP"] = df["QUEUED_TIMESTAMP"].apply(date_to_unix_seconds)
df["START_TIMESTAMP"] = df["START_TIMESTAMP"].apply(date_to_unix_seconds)

In [53]:
df["wait_time"] = df["START_TIMESTAMP"] - df["QUEUED_TIMESTAMP"]

In [54]:
df["EXIT_STATUS"].unique()

array([      0,     255,     143,       2,     134,       1,     137,
           127,     139,     128,     132, 1234567,     233,       5,
           136,     250,     126,     102,     213,       6,     135])

In [55]:
def exitcode_to_words(exit_code):
  if exit_code == 0:
    return "Pass"
  elif exit_code in [137, 143]:
    return "Killed"
  else:
    return "Failed"

df["EXIT_STATUS"] = df["EXIT_STATUS"].apply(exitcode_to_words)

In [56]:
df.head()

,JOB_NAME,USERNAME_GENID,PROJECT_NAME_GENID,state,CORES_USED,cpu_num,NODES_USED,QUEUED_TIMESTAMP,START_TIMESTAMP,RUNTIME_SECONDS,WALLTIME_SECONDS,EXIT_STATUS,wait_time
0,10120170,2.416635e+13,3.351006e+13,0,1.0,0,1.0,1.672529e+09,1.672529e+09,2305.54,3600.0,Pass,5.0
1,10120171,0.000000e+00,0.000000e+00,0,1.0,0,1.0,1.672529e+09,1.672529e+09,3602.30,3600.0,Failed,6.0
2,10120172,2.416635e+13,3.351006e+13,0,1.0,0,1.0,1.672532e+09,1.672532e+09,2300.64,3600.0,Pass,1.0
3,10120173,2.416635e+13,3.351006e+13,0,1.0,0,1.0,1.672535e+09,1.672535e+09,2307.36,3600.0,Pass,1.0
4,10120174,0.000000e+00,0.000000e+00,0,1.0,0,1.0,1.672535e+09,1.672535e+09,3603.24,3600.0,Failed,0.0


In [57]:
df["node_hour"] = df["NODES_USED"]*df["RUNTIME_SECONDS"]

In [58]:
df["node_hour"] = df["node_hour"]/3600

In [59]:
df.head()

,JOB_NAME,USERNAME_GENID,PROJECT_NAME_GENID,state,CORES_USED,cpu_num,NODES_USED,QUEUED_TIMESTAMP,START_TIMESTAMP,RUNTIME_SECONDS,WALLTIME_SECONDS,EXIT_STATUS,wait_time,node_hour
0,10120170,2.416635e+13,3.351006e+13,0,1.0,0,1.0,1.672529e+09,1.672529e+09,2305.54,3600.0,Pass,5.0,0.640428
1,10120171,0.000000e+00,0.000000e+00,0,1.0,0,1.0,1.672529e+09,1.672529e+09,3602.30,3600.0,Failed,6.0,1.000639
2,10120172,2.416635e+13,3.351006e+13,0,1.0,0,1.0,1.672532e+09,1.672532e+09,2300.64,3600.0,Pass,1.0,0.639067
3,10120173,2.416635e+13,3.351006e+13,0,1.0,0,1.0,1.672535e+09,1.672535e+09,2307.36,3600.0,Pass,1.0,0.640933
4,10120174,0.000000e+00,0.000000e+00,0,1.0,0,1.0,1.672535e+09,1.672535e+09,3603.24,3600.0,Failed,0.0,1.000900


In [60]:
df = df[["JOB_NAME", "USERNAME_GENID", "PROJECT_NAME_GENID", "state", "CORES_USED", "cpu_num", "NODES_USED", "QUEUED_TIMESTAMP", "wait_time", "RUNTIME_SECONDS", "WALLTIME_SECONDS", "node_hour", "EXIT_STATUS"]]

In [61]:
df.head()

,JOB_NAME,USERNAME_GENID,PROJECT_NAME_GENID,state,CORES_USED,cpu_num,NODES_USED,QUEUED_TIMESTAMP,wait_time,RUNTIME_SECONDS,WALLTIME_SECONDS,node_hour,EXIT_STATUS
0,10120170,2.416635e+13,3.351006e+13,0,1.0,0,1.0,1.672529e+09,5.0,2305.54,3600.0,0.640428,Pass
1,10120171,0.000000e+00,0.000000e+00,0,1.0,0,1.0,1.672529e+09,6.0,3602.30,3600.0,1.000639,Failed
2,10120172,2.416635e+13,3.351006e+13,0,1.0,0,1.0,1.672532e+09,1.0,2300.64,3600.0,0.639067,Pass
3,10120173,2.416635e+13,3.351006e+13,0,1.0,0,1.0,1.672535e+09,1.0,2307.36,3600.0,0.640933,Pass
4,10120174,0.000000e+00,0.000000e+00,0,1.0,0,1.0,1.672535e+09,0.0,3603.24,3600.0,1.000900,Failed


In [62]:
df.columns = ["job", "user", "project", "state", "gpu_num", "cpu_num", "node_num", "submit_time", "wait_time", "run_time", "wall_time", "node_hour", "new_status"]
df.head()

,job,user,project,state,gpu_num,cpu_num,node_num,submit_time,wait_time,run_time,wall_time,node_hour,new_status
0,10120170,2.416635e+13,3.351006e+13,0,1.0,0,1.0,1.672529e+09,5.0,2305.54,3600.0,0.640428,Pass
1,10120171,0.000000e+00,0.000000e+00,0,1.0,0,1.0,1.672529e+09,6.0,3602.30,3600.0,1.000639,Failed
2,10120172,2.416635e+13,3.351006e+13,0,1.0,0,1.0,1.672532e+09,1.0,2300.64,3600.0,0.639067,Pass
3,10120173,2.416635e+13,3.351006e+13,0,1.0,0,1.0,1.672535e+09,1.0,2307.36,3600.0,0.640933,Pass
4,10120174,0.000000e+00,0.000000e+00,0,1.0,0,1.0,1.672535e+09,0.0,3603.24,3600.0,1.000900,Failed


In [63]:
df.to_csv("data_thetagpu.csv")